In [43]:
import plotly.graph_objects as go
from pymongo import MongoClient, DESCENDING, ASCENDING
import pandas as pd
import numpy as np
import os, sys, time, datetime
import ta
import yfinance as yf

## Add New Time Interval

In [44]:
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['historic_data']
collection = db['daily_stock_price']
# Get one symbol data from the collection
df = pd.DataFrame(list(collection.find({'symbol':'NVDA'},{'_id': 0}).sort('date', ASCENDING)))
df

,date,symbol,open,low,high,close,volume
0,1999-01-22,NVDA,0.040125,0.035587,0.044783,0.037618,2714688000
1,1999-01-25,NVDA,0.040603,0.037618,0.042036,0.041559,510480000
2,1999-01-26,NVDA,0.042036,0.037737,0.042872,0.038334,343200000
3,1999-01-27,NVDA,0.038453,0.036303,0.039409,0.038215,244368000
4,1999-01-28,NVDA,0.038215,0.037856,0.038453,0.038095,227520000
...,...,...,...,...,...,...,...
6462,2024-09-27,NVDA,123.970001,119.260002,124.029999,121.400002,271009200
6463,2024-09-30,NVDA,118.309998,118.150002,121.500000,121.440002,226553700
6464,2024-10-01,NVDA,121.769997,115.790001,122.440002,117.000000,302094500
6465,2024-10-02,NVDA,116.440002,115.139999,119.379997,118.849998,221845900


In [46]:
new_interval_record = {}

def create_new_interval_df(df, interval):
    new_df = df.set_index('date')
    new_df = new_df.groupby(pd.Grouper(freq=interval)).agg({'open': 'first',
                                                'high': 'max', 
                                                'low': 'min', 
                                                'close': 'last', 
                                                'volume': 'sum'})
    new_df.reset_index(inplace=True)
    new_df = new_df.dropna()
    return new_df

create_new_interval_df(df,'1D')

,date,open,high,low,close,volume
0,1999-01-22,0.040125,0.044783,0.035587,0.037618,2714688000
3,1999-01-25,0.040603,0.042036,0.037618,0.041559,510480000
4,1999-01-26,0.042036,0.042872,0.037737,0.038334,343200000
5,1999-01-27,0.038453,0.039409,0.036303,0.038215,244368000
6,1999-01-28,0.038215,0.038453,0.037856,0.038095,227520000
...,...,...,...,...,...,...
9380,2024-09-27,123.970001,124.029999,119.260002,121.400002,271009200
9383,2024-09-30,118.309998,121.500000,118.150002,121.440002,226553700
9384,2024-10-01,121.769997,122.440002,115.790001,117.000000,302094500
9385,2024-10-02,116.440002,119.379997,115.139999,118.849998,221845900


## Add Technical Features

In [8]:
# Add bull and bear features in the dataframe
class add_features:
    def __init__(self, df):
        self.df = df.copy()

    def add_candlestick(self):
        self.df["BodyDiff"] = abs(self.df["open"] - self.df["close"])
        self.df["CandleStickType"] = np.where(self.df["open"] < self.df["close"], "green", "red")
        return self.df

    def continuous_increase(self, windows=3):
        for i in range(1, windows + 1):
            self.df[f"close_t-{i}"] = self.df["close"].shift(i)

        self.df['Incremental_High'] = (self.df['close'] > self.df['close_t-1']) \
                                        & (self.df['close_t-1'] > self.df['close_t-2']) \
                                        & (self.df['close_t-2'] > self.df['close_t-3'])
        return self.df

    def macd_golden_cross(self):
        self.df['MACD_GOLDEN_CROSS'] = (self.df['MACD'] > self.df['MACD_SIGNAL']) & (self.df['MACD'] < 0)
        return self.df

    def add_ema_band(self, threshold=0.05):
        self.df['169EMA_Upper'] = self.df['169EMA'] * (1 + threshold)
        self.df['169EMA_Lower'] = self.df['169EMA'] * (1 - threshold)
        return self.df
    
    # Add Technical Indicators
    def add_technical(self):
        
        # Add ema dual channels technical indicators
        self.df['8EMA'] = ta.trend.ema_indicator(self.df['close'], window=8)
        self.df['13EMA'] = ta.trend.ema_indicator(self.df['close'], window=13)
        self.df['144EMA'] = ta.trend.ema_indicator(self.df['close'], window=144)
        self.df['169EMA'] = ta.trend.ema_indicator(self.df['close'], window=169)

        # Add MACD technical indicator
        self.df['MACD'] = ta.trend.macd(self.df['close'], 
                                        window_slow=26, 
                                        window_fast=12)
        
        self.df['MACD_SIGNAL'] = ta.trend.macd_signal(self.df['close'], 
                                                    window_slow=26, 
                                                    window_fast=12, 
                                                    window_sign=9)
        
        self.df['MACD_HIST'] = ta.trend.macd_diff(self.df['close'], 
                                                window_slow=26, 
                                                window_fast=12, 
                                                window_sign=9)
        
        # Add ATR technical indicator
        self.df["atr"] = ta.volatility.AverageTrueRange(high=self.df.high, 
                                                        low=self.df.low, 
                                                        close=self.df.close).average_true_range()
        
        self.df["atr"] = self.df.atr.rolling(window=30).mean()
        
        return self.df
    
    def apply(self):
        self.add_technical()
        self.add_candlestick()
        self.continuous_increase()
        self.macd_golden_cross()
        self.add_ema_band()
        
        # Use last 1000 rows of the dataframe
        self.df = self.df.tail(3000) if len(self.df) > 3000 else self.df
        return self.df
    
testing_df = add_features(df).apply()

## Apply Strategy

In [9]:
class Alert:
    
    def __init__(self, df):
        self.df = df.copy()
        self.df['MACD_Alert'] = -1  
        self.df['Engulf_Alert'] = -1 
        self.df['dual_channel_Alert'] = -1  
        self.df['382_Alert'] = -1  
        self.window = 3  # Number of days to compare the stock price
        
    def engulf_alert(self):
        
        # Previous Candle
        prev_open = self.df['open'].shift(1)
        prev_close = self.df['close'].shift(1)
        
        # Current Candle
        current_open = self.df['open']
        current_close = self.df['close']

        # Bullish Engulfing: current green candle engulfs previous red candle
        bullish_engulfing = (
            (prev_close < prev_open) &  # Previous candle was red
            (current_close > current_open) &  # Current candle is green
            (current_open < prev_close) & 
            (current_close > prev_open)
        )

        # Bearish Engulfing: current red candle engulfs previous green candle
        bearish_engulfing = (
            (prev_close > prev_open) &  # Previous candle was green
            (current_close < current_open) &  # Current candle is red
            (current_open > prev_close) & 
            (current_close < prev_open)
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_engulfing, 'Engulf_Alert'] = 1
        self.df.loc[bearish_engulfing, 'Engulf_Alert'] = 0
        
        return self.df

    def macd_alert(self):
        
        # Pre-compute conditions
        macd_above_signal = self.df['MACD'] > self.df['MACD_SIGNAL']
        macd_increasing = self.df['MACD'].diff() > 0
        macd_below_zero = self.df['MACD'] >= 0

        # Bullish condition
        bullish_macd = (
            macd_above_signal &
            macd_increasing&
            macd_below_zero
        )
        
        # Bearish condition
        bearish_macd = (
            (self.df['MACD'] < self.df['MACD_SIGNAL']) &
            (self.df['MACD'] > self.df['MACD'].shift(-1))  
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_macd, 'MACD_Alert'] = 1
        self.df.loc[bearish_macd, 'MACD_Alert'] = 0

        return self.df

    def dual_channel(self):
        
        # Confirm Technical Indicators Conditions
        ema_8_gt_ema_13 = self.df['8EMA'] > self.df['13EMA']
        ema_13_gt_ema_169 = self.df['13EMA'] > self.df['169EMA']
        ema_8_gt_ema_144 = self.df['8EMA'] > self.df['144EMA']
        
        # Confirm Close price action
        close_ge_ema_13 = self.df['close'] >= self.df['13EMA']
        close_ge_ema_144 = self.df['close'] >= self.df['144EMA']
        
        # Confirm Candle fully above 13 EMA
        low_ge_ema_13 = self.df['low'] >= self.df['13EMA'] 
        green_candle = self.df['CandleStickType'] == 'green'    
        
        # Confirm Candle in selected range
        open_in_slow_ema = self.df['open'].between(self.df['169EMA_Lower'], self.df['169EMA_Upper'])
        
        # Confirm Candle type
        green_candle = self.df['CandleStickType'] == 'green'
        
        # Bullish Conditions
        
        bullish_scenario_1 = (
            ema_8_gt_ema_13 & 
            ema_13_gt_ema_169 & 
            close_ge_ema_13 &
            low_ge_ema_13 &
            green_candle
        )
        
        # Bearish Condition
        bearish_scenario_1 = (
            (self.df['open'] < self.df['13EMA']) &
            (self.df['close'] < self.df['13EMA']) &
            (self.df['13EMA'] < self.df['8EMA'])
        )
        # Bullish Conditions 2
        bullish_scenario_2 = ( 
            ema_8_gt_ema_144 &
            ema_13_gt_ema_169 &
            close_ge_ema_144 &
            open_in_slow_ema &
            green_candle
            )
        
        # Bearish Condition 2
        bearish_scenario_2 = (
            (self.df['open'] < self.df['13EMA']) &
            (self.df['close'] < self.df['13EMA']) &
            (self.df['8EMA'] < self.df['13EMA']) &
            ~ open_in_slow_ema
        )

        # Apply conditions on different price scenarios
        
        self.df.loc[bullish_scenario_1 | bullish_scenario_2, 'dual_channel_Alert'] = 1
        self.df.loc[bearish_scenario_1 | bearish_scenario_2, 'dual_channel_Alert'] = 0
        
        return self.df

    def bullish_382_alert(self):
        # Calculate Fibonacci 38.2% retracement level
        diff = self.df['high'] - self.df['low']
        fib_382_level = self.df['high'] - 0.382 * diff
        open_above_fib_382 = self.df['open'] > fib_382_level
        
        # Apply condition
        self.df.loc[open_above_fib_382, '382_Alert'] = 1
        self.df.loc[~open_above_fib_382, '382_Alert'] = 0
        
        return self.df

    def add_alert(self):
        self.engulf_alert()
        self.macd_alert()
        self.dual_channel()
        self.bullish_382_alert()
        return self.df
    
# Apply the strategy
testing_df = Alert(testing_df).add_alert()

print(f"Bullish MACD Count: {len(testing_df[testing_df['MACD_Alert'] == 1])} | Bearish Count: {len(testing_df[testing_df['MACD_Alert'] == 0])}")
print(f"Bullish Engulf Count: {len(testing_df[testing_df['Engulf_Alert'] == 1])} | Bearish Engulf Count: {len(testing_df[testing_df['Engulf_Alert'] == 0])}")
print(f"Bullish 382 Count: {len(testing_df[testing_df['382_Alert'] == 1])} | Bearish 382 Count: {len(testing_df[testing_df['382_Alert'] == 0])}")
print(f"Bullish Dual Channel Count: {len(testing_df[testing_df['dual_channel_Alert'] == 1])} | Bearish Dual Channel Count: {len(testing_df[testing_df['dual_channel_Alert'] == 0])}")


Bullish MACD Count: 829 | Bearish Count: 973
Bullish Engulf Count: 138 | Bearish Engulf Count: 90
Bullish 382 Count: 965 | Bearish 382 Count: 2035
Bullish Dual Channel Count: 928 | Bearish Dual Channel Count: 454


In [10]:
testing_df

,date,symbol,low,open,close,high,volume,8EMA,13EMA,144EMA,...,close_t-2,close_t-3,Incremental_High,MACD_GOLDEN_CROSS,169EMA_Upper,169EMA_Lower,MACD_Alert,Engulf_Alert,dual_channel_Alert,382_Alert
3540,2012-11-01,GIB,25.559999,26.000000,26.330000,26.330000,145400,26.014459,26.122552,24.356231,...,25.260000,26.150000,False,False,25.198820,22.798933,-1,-1,-1,0
3541,2012-11-02,GIB,26.070000,26.350000,26.139999,26.389999,78100,26.042357,26.125044,24.380835,...,26.100000,25.260000,False,False,25.225269,22.822863,-1,-1,-1,1
3542,2012-11-05,GIB,26.070000,26.100000,26.080000,26.389999,66100,26.050722,26.118610,24.404272,...,26.330000,26.100000,False,False,25.250666,22.845841,0,-1,0,0
3543,2012-11-06,GIB,24.639999,26.120001,24.850000,26.120001,310900,25.783895,25.937380,24.410420,...,26.139999,26.330000,False,False,25.260570,22.854802,0,-1,-1,1
3544,2012-11-07,GIB,24.660000,24.740000,24.680000,25.010000,219500,25.538585,25.757754,24.414138,...,26.080000,26.139999,False,False,25.268258,22.861757,0,-1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2024-09-27,GIB,114.199997,116.669998,114.220001,116.669998,107000,114.386655,114.062882,107.774938,...,113.820000,114.529999,False,False,112.684220,101.952390,0,0,-1,1
6536,2024-09-30,GIB,113.010002,114.059998,114.959999,115.279999,83900,114.514065,114.191041,107.874042,...,116.400002,113.820000,False,False,112.778618,102.037797,0,-1,-1,0
6537,2024-10-01,GIB,113.400002,115.000000,114.320000,115.279999,93400,114.470939,114.209464,107.962952,...,114.220001,116.400002,False,False,112.863999,102.115046,0,-1,-1,1
6538,2024-10-02,GIB,113.730003,113.730003,115.129997,115.889999,133000,114.617397,114.340969,108.061808,...,114.959999,114.220001,False,False,112.958381,102.200440,-1,1,-1,0
